In [47]:
from sumpy.recurrence import _make_sympy_vec, get_processed_and_shifted_recurrence

from sumpy.expansion.diff_op import (
    laplacian,
    make_identity_diff_op,
)

from sumpy.recurrence import get_recurrence

import sympy as sp
import numpy as np

In [48]:
w = make_identity_diff_op(2)
laplace2d = laplacian(w)
n_init, order, r = get_processed_and_shifted_recurrence(laplace2d)

In [49]:
def scale_recurrence(r):
    #We want to subsitute s(i) r^i_{ct} = g(i)
    g = sp.Function("g")
    s = sp.Function("s")
    n = sp.symbols("n")
    rct = sp.symbols("r_{ct}")

    r_new = r*rct**n
    for i in range(order):
        r_new = r_new.subs(s(n-i),g(n-i)/(rct**(n-i)))

    return r_new

In [50]:
r_new = scale_recurrence(r)

In [51]:
max_abs = .0000001
var = _make_sympy_vec("x", 2)
rct = sp.symbols("r_{ct}")
g = sp.Function("g")
n = sp.symbols("n")

In [52]:
r_new

(-1)**(n + 1)*r_{ct}**n*((-1)**(n - 3)*r_{ct}**(3 - n)*(n + (n - 2)**3 - 2*(n - 2)**2 - 2)*g(n - 3)/(x0**3 + x0*x1**2) + (-1)**(n - 2)*r_{ct}**(2 - n)*(-n + 3*(n - 2)**2 + 2)*g(n - 2)/(x0**2 + x1**2) + (-1)**(n - 1)*r_{ct}**(1 - n)*(3*x0**2*(n - 2) + x0**2 + x1**2*(n - 2) - x1**2)*g(n - 1)/(x0**3 + x0*x1**2))

In [53]:
def compute_derivatives(p):
    var = _make_sympy_vec("x", 2)
    var_t = _make_sympy_vec("t", 2)
    g_x_y = sp.log(sp.sqrt((var[0]-var_t[0])**2 + (var[1]-var_t[1])**2))
    derivs = [sp.diff(g_x_y,
                        var_t[0], i).subs(var_t[0], 0).subs(var_t[1], 0)
                        for i in range(p)]
    return derivs
derivs = compute_derivatives(15)

In [54]:
def evaluate_recurrence(coord_dict, rct_val, recur, p):
    subs_dict = {}
    subs_dict[g(0)] = derivs[0].subs(coord_dict).subs(rct, rct_val)
    subs_dict[g(1)] = derivs[1].subs(coord_dict).subs(rct, rct_val) * rct_val
    var = _make_sympy_vec("x", 2)
    for i in range(2, p):
        subs_dict[g(i)] = get_recurrence(recur.subs(rct, rct_val), i).subs(subs_dict).subs(coord_dict)
        print(get_recurrence(recur.subs(rct, 1),i))
    return np.array(list(subs_dict.values()))

In [55]:
def evaluate_recurrence_lamb(coord_dict, rct_val, recur, p):
    subs_dict = {}
    subs_dict[g(-2)] = 0
    subs_dict[g(-1)] = 0
    subs_dict[g(0)] = derivs[0].subs(coord_dict).subs(rct, rct_val)
    subs_dict[g(1)] = derivs[1].subs(coord_dict).subs(rct, rct_val) * rct_val
    var = _make_sympy_vec("x", 2)
    for i in range(2, p):
        exp = get_recurrence(recur.subs(rct, rct_val), i)
        f = sp.lambdify([var[0], var[1], g(i-1), g(i-2), g(i-3)], exp)
        subs_dict[g(i)] = f(coord_dict[var[0]], coord_dict[var[1]], subs_dict[g(i-1)],
                            subs_dict[g(i-2)], subs_dict[g(i-3)])
    subs_dict.pop(g(-2))
    subs_dict.pop(g(-1))
    return np.array(list(subs_dict.values()))

In [56]:
def evaluate_true(coord_dict, rct_val, p):
    retMe = []
    for i in range(p):
        exp = (derivs[i]*rct_val**i)
        f = sp.lambdify(var, exp)
        retMe.append(f(coord_dict[var[0]], coord_dict[var[1]]))
    return np.array(retMe)

In [91]:
def compute_error(pw):
    x_coord = 1e-7
    y_coord = 1
    var = _make_sympy_vec("x", 2)
    coord_dict = {var[0]: x_coord, var[1]: y_coord}

    rct_val = 1
    exp = evaluate_recurrence_lamb(coord_dict, rct_val, r_new, 14)
    true = evaluate_true(coord_dict, rct_val, 14)
    print(exp)
    print(true)
    return np.abs(exp-true)/np.abs(true)

In [92]:
compute_error(1)

[4.88498130835068e-15 -9.99999999999990e-8 0.999999999999970
 5.99999999999980e-7 -5.99999999999940 -1.20103359222412e-5
 119.689922332713 -12403106.6899557 -620155334528027.
 -3.72093200713792e+22 -2.60465240499655e+30 -2.08372192399724e+38
 -1.87534973159752e+46 -1.87534973159752e+54]
[ 4.88498131e-15 -1.00000000e-07  1.00000000e+00  6.00000000e-07
 -6.00000000e+00 -1.20000000e-05  1.20000000e+02  5.04000000e-04
 -5.04000000e+03 -3.62880000e-02  3.62880000e+05  3.99168000e+00
 -3.99168000e+07 -6.22702080e+02]


array([0, 0, 1.11022302462519e-16, 0, 4.44089209850107e-16,
       0.000861326853504301, 0.00258398056051277, 24609338671.5500,
       123046693357.780, 1.02538911131465e+24, 7.17772377920519e+24,
       5.22016274851136e+37, 4.69814647366267e+38, 3.01163235491067e+51],
      dtype=object)

In [104]:
a = sp.cancel(sp.diff(r_new.subs(rct, var[0]).subs(n, 5), var[0], 0))
a

(12*x0**2*g(2) - 24*x0**2*g(3) + 10*x0**2*g(4) + 2*x1**2*g(4))/(x0**2 + x1**2)

In [110]:
sp.diff(a, var[0], 6).subs(var[0], 0)

1440*(6*g(2) - 12*g(3) + 4*g(4))/x1**6

In [111]:
sp.diff(a, var[0], 6).subs(var[0], 0).subs(g(2), 1.00000000e+00).subs(g(3), 6.00000000e-07).subs(g(4), -6.00000000e+00)

-25920.010368/x1**6

In [34]:
(30116417561.29867 -30116417561.298637)/30116417561.298637

1.1399853691344745e-15

In [13]:
pw = 8
x0 = 1
x1 = 10**(-pw)
g_1 = -5000000000.00000
g_2 = (x0**2 - x1**2)*g_1/(x0**3 + x0*x1**2)
g_2

-4999999999.999999

In [14]:
derivs[7]

-114.591559026165*x0*(64*x0**6/(x0**2 + x1**2)**3 - 112*x0**4/(x0**2 + x1**2)**2 + 56*x0**2/(x0**2 + x1**2) - 7)/(x0**2 + x1**2)**4

In [15]:
derivs[2].subs(var[0], 10**(-8)).subs(var[1], 10**(-8))

0.0883487411517643

In [16]:
g_2

-4999999999.999999

In [17]:
'''
x_plot = [i for i in range(len(compute_error(0)))]
for i in range(1, 4):
    plt.semilogy(x_plot, compute_error(i), label=str(10**(-i)))
plt.xlabel("order of derivative being computed")
plt.ylabel("absolute error")
plt.title("recurrence error vs order for different source-locations")
plt.legend(title='ratio of x_{coord_src}/y_{coord_src}')
plt.show()
'''

'\nx_plot = [i for i in range(len(compute_error(0)))]\nfor i in range(1, 4):\n    plt.semilogy(x_plot, compute_error(i), label=str(10**(-i)))\nplt.xlabel("order of derivative being computed")\nplt.ylabel("absolute error")\nplt.title("recurrence error vs order for different source-locations")\nplt.legend(title=\'ratio of x_{coord_src}/y_{coord_src}\')\nplt.show()\n'